## ReadMe

This notebook creates a summary table from a list of files and columns that are the direct output of CCDR analytic tool.

In [1]:
# Importing all the dependencies
import datetime, os, shutil, gc, socket, glob
from tqdm.notebook import tqdm
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# Defining the initial parameters
sISO = 'KHM'  # Select here the country code of analysis
sADM = 'ADM3' # Select here the ADM level of analysis
sHaz = ['AP', 'ASI_S1_30perc', 'ASI_S1_50perc', 'CF', 'FL', 'HS', 'LS', 'TC'] # Select here the hazards for summary
sExp_agri    = 'ESA20_agri'
sExp_builtup = 'WSF19'
sExp_pop     = 'WPOP20'

In [5]:
# Retrieving the list of ,gpkg files to process
sFiles = glob.glob('./*.gpkg')
sFiles = [s for s in sFiles if sISO in s]
sFiles = [s for s in sFiles if sADM in s]

# Defining the list of column names and description for the ADM data
iADM = int(sADM[len(sADM)-1])
sADMcols = ['ADM'+str(i)+'_CODE' for i in range(iADM+1)]
sADMcols = sADMcols + ['ADM'+str(i)+'_NAME' for i in range(iADM+1)]
sADMcolsDesc = ['ADM'+str(i)+' unique identifier code' for i in range(iADM+1)]
sADMcolsDesc = sADMcolsDesc + ['ADM'+str(i)+' unit name' for i in range(iADM+1)]

# Defining the list of files to extract info for summary (the item order of sFileGet, sFileGetCols, and sFileGetColsDesc must match)
sFileGet = [
    sISO+'_'+sADM+'_'+'AP_pop_'+sExp_pop+'_class.gpkg', 
    sISO+'_'+sADM+'_'+'ASI_S1_30perc_agri_'+sExp_agri+'_function.gpkg', 
    sISO+'_'+sADM+'_'+'ASI_S1_50perc_agri_'+sExp_agri+'_function.gpkg',
    sISO+'_'+sADM+'_'+'ASI_S2_30perc_agri_'+sExp_agri+'_function.gpkg',
    sISO+'_'+sADM+'_'+'ASI_S2_50perc_agri_'+sExp_agri+'_function.gpkg',
    sISO+'_'+sADM+'_'+'CF_agri_'+sExp_agri+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'CF_builtup_'+sExp_builtup+'_EAE_class.gpkg', 
    sISO+'_'+sADM+'_'+'CF_builtup_'+sExp_builtup+'_EAI_function.gpkg',
    sISO+'_'+sADM+'_'+'CF_pop_'+sExp_pop+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'CF_pop_'+sExp_pop+'_EAI_function.gpkg',
    sISO+'_'+sADM+'_'+'FL_agri_'+sExp_agri+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'FL_builtup_'+sExp_builtup+'_EAE_class.gpkg', 
    sISO+'_'+sADM+'_'+'FL_builtup_'+sExp_builtup+'_EAI_function.gpkg',
    sISO+'_'+sADM+'_'+'FL_pop_'+sExp_pop+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'FL_pop_'+sExp_pop+'_EAI_function.gpkg',
    sISO+'_'+sADM+'_'+'HS_pop_'+sExp_pop+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'EQ_builtup_'+sExp_builtup+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'EQ_pop_'+sExp_pop+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'LS_builtup_'+sExp_builtup+'_class.gpkg',
    sISO+'_'+sADM+'_'+'LS_pop_'+sExp_pop+'_class.gpkg',
    sISO+'_'+sADM+'_'+'TC_agri_'+sExp_agri+'_EAE_class.gpkg',
    sISO+'_'+sADM+'_'+'TC_builtup_'+sExp_builtup+'_EAE_class.gpkg', 
    sISO+'_'+sADM+'_'+'TC_builtup_'+sExp_builtup+'_EAI_function.gpkg',
    sISO+'_'+sADM+'_'+'TC_pop_'+sExp_pop+'_EAE_class.gpkg',
]

# Defining the list of columns (matching in order sFilesGet) to extract info for summary
sFileGetCols = [
    'AP_pop_'+sExp_pop+'_C2',               # AP - Air Pollution - High hazard 15-25 μg/m3 - 16% increased mortality rate
    'DR.S1.30p_agri_'+sExp_agri+'_freq',    # ASI_S1_30perc - Drought
    'DR.S1.50p_agri_'+sExp_agri+'_freq',    # ASI_S1_50perc - Drought
    'DR.S2.30p_agri_'+sExp_agri+'_freq',    # ASI_S2_30perc - Drought
    'DR.S2.50p_agri_'+sExp_agri+'_freq',    # ASI_S2_50perc - Drought
    'CF_agri_'+sExp_agri+'_C0_EAE',         # CF - Coastal Flood - Over 5cm of flood depth
    'CF_builtup_'+sExp_builtup+'_C0_EAE',   # CF - Coastal Flood - Over 5cm of flood depth
    'CF_builtup_'+sExp_builtup+'_EAI',      # CF - Coastal Flood - Impact on builtup area
    'CF_pop_'+sExp_pop+'_C0_EAE',           # CF - Coastal Flood - Over 5cm of flood depth
    'CF_pop_'+sExp_pop+'_EAI',              # CF - Coastal Flood - Impact on population (mortality)
    'FL_agri_'+sExp_agri+'_C0_EAE',         # FL - River Flood - Over 5cm of flood depth
    'FL_builtup_'+sExp_builtup+'_C0_EAE',   # FL - River Flood - Over 5cm of flood depth
    'FL_builtup_'+sExp_builtup+'_EAI',      # FL - River Flood - Impact on builtup area
    'FL_pop_'+sExp_pop+'_C0_EAE',           # FL - River Flood - Over 5cm of flood depth
    'FL_pop_'+sExp_pop+'_EAI',              # FL - River Flood - Impact on population (mortality)
    'HS_pop_'+sExp_pop+'_C0_EAE',           # HS - Heat Stress - Very strong hazard - WBGT (°C) 28-30°
    'EQ_builtup_'+sExp_builtup+'_C0_EAE',   # EQ - Earthquake - Strong perceived shaking - Velocity 9.64-20.00 cm/s - Light potential damage
    'EQ_pop_'+sExp_pop+'_C0_EAE',           # EQ - Earthquake - Strong perceived shaking - Velocity 9.64-20.00 cm/s - Light potential damage
    'LS_builtup_'+sExp_builtup+'_C2',       # LS - Landslide - High hazard - ARUP Wet LS > 0.01
    'LS_pop_'+sExp_pop+'_C2',               # LS - Landslide - High hazard - ARUP Wet LS > 0.01
    'TC_agri_'+sExp_agri+'_C3_EAE',         # TC - Tropical Cyclone - Category 3 Saffir–Simpson scale - Maximum sustained wind speed > 50 m/s
    'TC_builtup_'+sExp_builtup+'_C3_EAE',   # TC - Tropical Cyclone - Category 3 Saffir–Simpson scale - Maximum sustained wind speed > 50 m/s
    'TC_builtup_'+sExp_builtup+'_EAI',      # TC - Tropical Cyclone - Impact on builtup area
    'TC_pop_'+sExp_pop+'_C3_EAE',           # TC - Tropical Cyclone - Category 3 Saffir–Simpson scale - Maximum sustained wind speed > 50 m/s
]


# Defining the description of the list of columns (matching in order sFilesGet) to extract info for summary
sFileGetColsDesc = [
    'Population exposed to high air pollution hazard (population count and relative)',
    'Frequency of severe agricultural drought affecting at least 30% of arable land during growing season 1 (percentage of historical period 1984-2022)',
    'Frequency of severe agricultural drought affecting at least 50% of arable land during growing season 1 (percentage of historical period 1984-2022)',
    'Frequency of severe agricultural drought affecting at least 30% of arable land during growing season 2 (percentage of historical period 1984-2022)',
    'Frequency of severe agricultural drought affecting at least 50% of arable land during growing season 2 (percentage of historical period 1984-2022)',
    'Expected annual exposure of agricultural land to coastal floods (hectares and relative)', 
    'Expected annual exposure of builtup area to coastal floods (hectares and relative)', 
    'Expected annual impact on builtup area due to coastal floods (hectares and relative)', 
    'Expected annual exposure of population to coastal floods (population count and relative)', 
    'Expected annual mortality due to coastal floods (population count and relative)', 
    'Expected annual exposure of agricultural land to river floods (hectares and relative)', 
    'Expected annual exposure of builtup area to river floods (hectares and relative)', 
    'Expected annual impact on builtup area due to river floods (hectares and relative)', 
    'Expected annual exposure of population to river floods (population count and relative)', 
    'Expected annual mortality due to river floods (population count and relative)', 
    'Expected annual exposure of population to very strong heat stress hazard (population count and relative)', 
    'Expected annual exposure of builtup area to strong perceived shaking earthquake hazard (hectares and relative)', 
    'Expected annual exposure of population to strong perceived shaking earthquake hazard (population count and relative)', 
    'Builtup area exposed to high landslide hazard (hectares and relative)',
    'Population exposed to high landslide hazard (population count and relative)',
    'Expected annual exposure of agricultural land to tropical cyclone category 3 on the Saffir–Simpson scale (hectares and relative)', 
    'Expected annual exposure of builtup area to tropical cyclone category 3 on the Saffir–Simpson scale (hectares and relative)', 
    'Expected annual impact on builtup area due to tropical cyclone (hectares and relative)', 
    'Expected annual exposure of population to tropical cyclone category 3 on the Saffir–Simpson scale (hectares and relative)', 
]

# Filtering sHaz in sFileGet
sFileGetFilter = []
for sHazStr in sHaz: sFileGetFilter = sFileGetFilter + [s for s in sFileGet if sHazStr in s]
sFileGetFilterIndex = [i for i, val in enumerate(sFileGet) if val in sFileGetFilter]

# Filtering the list of column names and description based on the filter of files
sFileGetColsFilter = [sFileGetCols[i] for i in sFileGetFilterIndex]
sFileGetColsDescFilter = [sFileGetColsDesc[i] for i in sFileGetFilterIndex]

# Creating the summary description tab as a pandas data.frame
sSummaryDesc = {'Column Name':['Summary']+sADMcols+sFileGetColsFilter, 
                'Description':[' ']+sADMcolsDesc+sFileGetColsDescFilter}
dfSummaryDesc = pd.DataFrame(sSummaryDesc)

# Reading the dummy gpd file to store data into
gpdData = gpd.read_file(sFiles[0])
gpdData = gpdData[['geometry']+sADMcols]

# Adding the agri, builtup, and pop ADM values
sFileAgri = [s for s in sFiles if 'agri' in s][0]
gpdDataAgri = gpd.read_file(sFileAgri)[['geometry']+sADMcols+[sADM+'_agri_'+sExp_agri]]
gpdData = gpdData.merge(gpdDataAgri, how='left', on=['geometry']+sADMcols)
sFileBuiltup = [s for s in sFiles if 'builtup' in s][0]
gpdDataBuiltup = gpd.read_file(sFileBuiltup)[['geometry']+sADMcols+[sADM+'_builtup_'+sExp_builtup]]
gpdData = gpdData.merge(gpdDataBuiltup, how='left', on=['geometry']+sADMcols)
sFilePop = [s for s in sFiles if 'pop' in s][0]
gpdDataPop = gpd.read_file(sFilePop)[['geometry']+sADMcols+[sADM+'_pop_'+sExp_pop]]
gpdData = gpdData.merge(gpdDataPop, how='left', on=['geometry']+sADMcols)
del(sFileAgri,sFileBuiltup,sFilePop,gpdDataAgri,gpdDataBuiltup,gpdDataPop)

# For every file in sFileGetFilter, retrieve the desired columns in sFileGetColsFilter for the original ADM level
for i in range(len(sFileGetFilter)):
    sFileGetFilterOpen = sFileGetFilter[i]
    gpdDataOpen = gpd.read_file(sFileGetFilterOpen)
    sFileGetColsNames = sFileGetColsFilter[i]
    sFileGetColsOpen = sFileGetColsNames.split('_')
    sFileGetColsOpen = sFileGetColsOpen[1:len(sFileGetColsOpen)]
    sFileGetColsOpen = '_'.join(sFileGetColsOpen)
    if sFileGetColsOpen+'%' in gpdDataOpen.columns: 
        sFileGetColsNames = [sFileGetColsNames] + [sFileGetColsNames+'%']
        sFileGetColsOpen = [sFileGetColsOpen] + [sFileGetColsOpen+'%']
    else:
        sFileGetColsNames = [sFileGetColsNames]
        sFileGetColsOpen = [sFileGetColsOpen]
    gpdDataOpen = gpdDataOpen[['geometry']+sADMcols+sFileGetColsOpen]
    gpdDataColNames = list(gpdData.columns) + sFileGetColsNames
    gpdData = gpdData.merge(gpdDataOpen, how='left', on=['geometry']+sADMcols)
    gpdData.columns = gpdDataColNames
dfData = pd.DataFrame(gpdData)
dfData = dfData.drop(columns=['geometry'])

# Creating a single Excel output file with all .csv information
dfOverview = pd.DataFrame(['Overview'])
with pd.ExcelWriter(sISO+'_'+sADM+'_RSK.xlsx') as writer: 
    print('Creating a single Excel output file with all information')
    # Creating the legend overview and summary at ADM level tabs (overview is empty and needs to be manually updated later in Excel)
    dfSummaryDesc.to_excel(writer, sheet_name='Legend')
    dfOverview.to_excel(writer, sheet_name='Overview')
    dfData.to_excel(writer, sheet_name=sADM+'_Summary')
    # Computing the aggregate ADM results, from the original ADM level up to ADM0
    print('Writing the aggregate ADM results, from the original ADM level up to ADM0')
    dfDataAgg = dfData.copy()
    for j in reversed(range(iADM)):
        # Getting the list of ADM codes to aggregate to
        sADMagg = 'ADM'+str(j)
        sADMcolsAgg = ['ADM'+str(i)+'_CODE' for i in range(j+1)]
        sADMcolsAgg = sADMcolsAgg + ['ADM'+str(i)+'_NAME' for i in range(j+1)]
        # Computing the weighted average for drought (DR) related variables
        sColDR = [s for s in list(dfDataAgg.columns) if 'DR.S' in s]
        for iCol in range(len(sColDR)):
            if iCol == 0:
                dfDRtmp = dfDataAgg.groupby(sADMcolsAgg, as_index=False).apply(lambda x: np.average(x[sColDR[iCol]], weights=x['ADM'+str(j+1)+'_agri_'+sExp_agri]))
            else:
                dfDRtmp = dfDRtmp.merge(dfDataAgg.groupby(sADMcolsAgg, as_index=False).apply(lambda x: np.average(x[sColDR[iCol]], 
                                                                                                                  weights=x['ADM'+str(j+1)+'_agri_'+sExp_agri])), 
                                        how='left', on=sADMcolsAgg)
            dfDRtmpColNames = list(dfDRtmp.columns)
            dfDRtmpColNames[len(dfDRtmpColNames)-1] = sColDR[iCol]
            dfDRtmp.columns = dfDRtmpColNames
        # Aggregating the data by sum
        dfDataAgg = dfDataAgg.groupby(sADMcolsAgg, as_index=False).sum()
        dfDataAgg = dfDataAgg.drop(columns=['ADM'+str(j+1)+'_CODE'])
        dfDataAggCols = list(dfDataAgg.columns)
        dfDataAgg.columns = [s.replace('ADM'+str(j+1), sADMagg) for s in dfDataAggCols]
        # Applying the corrected weighted average for drought (DR) related variables
        dfDataAgg = dfDataAgg.drop(columns=sColDR)
        dfDataAgg = dfDataAgg.merge(dfDRtmp, how='left', on=sADMcolsAgg)
        # Computing the relative '%' values
        sColRelative = [s for s in list(dfDataAgg.columns) if '%' in s]
        sColRelative_agri    = [s for s in sColRelative if 'agri' in s]
        sColRelative_builtup = [s for s in sColRelative if 'builtup' in s]
        sColRelative_pop     = [s for s in sColRelative if 'pop' in s]
        for sCol in sColRelative_agri:    dfDataAgg[[sCol]] = dfDataAgg[[sCol.replace('%','')]].values/dfDataAgg[[sADMagg+'_agri_'+sExp_agri]].values
        for sCol in sColRelative_builtup: dfDataAgg[[sCol]] = dfDataAgg[[sCol.replace('%','')]].values/dfDataAgg[[sADMagg+'_builtup_'+sExp_builtup]].values
        for sCol in sColRelative_pop:     dfDataAgg[[sCol]] = dfDataAgg[[sCol.replace('%','')]].values/dfDataAgg[[sADMagg+'_pop_'+sExp_pop]].values
        # Creating the tabs for aggregate ADM levels, up to ADM0
        dfDataAgg.to_excel(writer, sheet_name=sADMagg+'_Summary')
    # Adding the original outputs from the analysis
    print('Adding the original single outputs from the analysis')
    for sFile in tqdm(sFileGetFilter):
        dfCSV = pd.read_csv(sFile.replace('.gpkg','.csv'))
        sSheet = sFile.split('/')
        sSheet = sSheet[len(sSheet)-1]
        sSheet = sSheet.replace('.gpkg','').replace('./'+sISO+'_','').replace(sADM+'_','')[:31]
        dfCSV.to_excel(writer, sheet_name=sSheet)
    # Saving results to disk
    print('Saving results to disk')
    writer.save()
print('Done!')

Creating a single Excel output file with all information
Writing the aggregate ADM results, from the original ADM level up to ADM0
Adding the original single outputs from the analysis


  0%|          | 0/20 [00:00<?, ?it/s]

Saving results to disk
Done!
